In [14]:
from rdkit import Chem
from rdkit.Chem import rdFingerprintGenerator

import numpy as np
import pandas as pd
import math

from collections import Counter

import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

from sklearn.metrics import root_mean_squared_error,r2_score,mean_absolute_error
from sklearn.model_selection import train_test_split, ParameterGrid, cross_val_score, cross_val_predict, LeaveOneOut
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import LeaveOneOut

from tqdm import tqdm

random_seed = 42

# Acyclic

In [15]:
df_acyclic = (pd.read_csv('../data/combined_data.csv', sep=';'))
df_acyclic['Mol'] = df_acyclic.smiles.apply(Chem.MolFromSmiles)

In [16]:
df_acyclic.head(2)

,name,smiles,InChIKey,molecular_formula,boiling_point,Mol
0,"1,1,1,2,2,3,4,5,5,6,6,6-dodecafluoro-3,4-bis(t...",C(C(C(C(F)(F)F)(F)F)(C(F)(F)F)F)(C(C(F)(F)F)(F...,AILNJPONTDNFHN-UHFFFAOYSA-N,C8F18,103.75,<rdkit.Chem.rdchem.Mol object at 0x000001FD6B7...
1,"1,1,1,2,2,3,3,4,4,5,5,6,6,7,7,8,8,9,9,10,10,11...",C(C(C(C(C(C(C(C(C(F)(F)F)(F)F)(F)F)(F)F)(F)F)(...,AQPUCGPFMVEJGS-UHFFFAOYSA-N,C16F34,239.00,<rdkit.Chem.rdchem.Mol object at 0x000001FD6B9...


In [17]:
r_list = [0, 1, 2, 3]
fpSize_list = [256, 512, 1024, 2048, 4096]

metrics_acyclic = {}
loo = LeaveOneOut()

for r in tqdm(r_list):
    for fpSize in tqdm(fpSize_list):
        MFPGEN = rdFingerprintGenerator.GetMorganGenerator(fpSize=fpSize, radius=r)
        X = np.stack(df_acyclic.Mol.apply(MFPGEN.GetCountFingerprintAsNumPy).values)
        y = df_acyclic['boiling_point'].values

        model = RandomForestRegressor(n_estimators=20, max_depth=3, random_state=random_seed)
        y_pred = cross_val_predict(model, X, y, cv=loo)
        
        rmse = root_mean_squared_error(y, y_pred)
        mae = mean_absolute_error(y, y_pred)
        r2 = r2_score(y, y_pred)

        metrics_acyclic[(r, fpSize)] = {
            'rmse': rmse,
            'mae': mae,
            'r2': r2,
            'y_true': y,
            'y_pred': y_pred
        }


100%|██████████| 4/4 [00:16<00:00,  4.12s/it]


In [19]:
def display_metrics_table(metrics_dict):
    rows = []
    for (r, fpSize), metrics in metrics_dict.items():
        row = {
            'Radius': r,
            'Fingerprint Size': fpSize,
            'RMSE': metrics['rmse'],
            'RMSE Std': metrics.get('rmse_std', float('nan')),
            'MAE': metrics['mae'],
            'MAE Std': metrics.get('mae_std', float('nan')),
            'R2': metrics.get('r2', float('nan'))
        }
        rows.append(row)
    
    df = pd.DataFrame(rows)
    
    min_rmse = df['RMSE'].min()
    min_mae = df['MAE'].min()
    max_r2 = df['R2'].max()
    
    def highlight_min(s, min_value):
        return ['background-color: green' if v == min_value else '' for v in s]
    
    def highlight_max(s, max_value):
        return ['background-color: green' if v == max_value else '' for v in s]

    styled_df = df.style\
        .apply(lambda s: highlight_min(s, min_rmse), subset=['RMSE'])\
        .apply(lambda s: highlight_min(s, min_mae), subset=['MAE'])\
        .apply(lambda s: highlight_max(s, max_r2), subset=['R2'])\
        .format("{:.4f}", subset=['RMSE', 'RMSE Std', 'MAE', 'MAE Std', 'R2'])

    return styled_df

In [13]:
display_metrics_table(metrics_acyclic)

,Radius,Fingerprint Size,RMSE,RMSE Std,MAE,MAE Std,R2
0,0,256,20.7930,nan,11.4486,nan,0.9331
1,0,512,20.7930,nan,11.4486,nan,0.9331
2,0,1024,20.7930,nan,11.4486,nan,0.9331
3,0,2048,20.7930,nan,11.4486,nan,0.9331
4,0,4096,20.7930,nan,11.4486,nan,0.9331
5,1,256,20.8695,nan,12.0865,nan,0.9326
6,1,512,20.6799,nan,11.2527,nan,0.9339
7,1,1024,20.7126,nan,11.7060,nan,0.9336
8,1,2048,20.5963,nan,12.0146,nan,0.9344
9,1,4096,20.7006,nan,11.4652,nan,0.9337


The fpSize does not matter, radius 0 is best

In [29]:
import sklearn
print(sklearn.__version__)


1.5.1


# OOD split

In [20]:
df_acyclic['atom_count'] = df_acyclic['Mol'].apply(lambda mol: sum(1 for atom in mol.GetAtoms() if atom.GetAtomicNum() == 6))
df_acyclic.head(2)

,name,smiles,InChIKey,molecular_formula,boiling_point,Mol,atom_count
0,"1,1,1,2,2,3,4,5,5,6,6,6-dodecafluoro-3,4-bis(t...",C(C(C(C(F)(F)F)(F)F)(C(F)(F)F)F)(C(C(F)(F)F)(F...,AILNJPONTDNFHN-UHFFFAOYSA-N,C8F18,103.75,<rdkit.Chem.rdchem.Mol object at 0x000001FD6B7...,8
1,"1,1,1,2,2,3,3,4,4,5,5,6,6,7,7,8,8,9,9,10,10,11...",C(C(C(C(C(C(C(C(C(F)(F)F)(F)F)(F)F)(F)F)(F)F)(...,AQPUCGPFMVEJGS-UHFFFAOYSA-N,C16F34,239.00,<rdkit.Chem.rdchem.Mol object at 0x000001FD6B9...,16


In [10]:
df_both['atom_count'] = df_both['Mol'].apply(lambda mol: sum(1 for atom in mol.GetAtoms() if atom.GetAtomicNum() == 6))
df_both.head(2)

,Mol,boiling_point,atom_count
0,<rdkit.Chem.rdchem.Mol object at 0x000001D0416...,103.75,8
1,<rdkit.Chem.rdchem.Mol object at 0x000001D0416...,239.00,16


In [21]:
df_sorted = df_acyclic.sort_values(by='atom_count')

split_index = int(len(df_sorted) * 0.85)

df_train = df_sorted.iloc[:split_index].reset_index(drop=True)
df_test = df_sorted.iloc[split_index:].reset_index(drop=True)

print("Train set size:", len(df_train))
print("Test set size:", len(df_test))

Train set size: 22
Test set size: 5


In [11]:
df_sorted = df_both.sort_values(by='atom_count')

split_index = int(len(df_sorted) * 0.85)

df_train = df_sorted.iloc[:split_index].reset_index(drop=True)
df_test = df_sorted.iloc[split_index:].reset_index(drop=True)

print("Train set size:", len(df_train))
print("Test set size:", len(df_test))

Train set size: 56
Test set size: 11


In [22]:
r_list = [0, 1, 2, 3]
fpSize_list = [256, 512, 1024, 2048, 4096]

metrics_ood= {}

for r in tqdm(r_list):
    for fpSize in tqdm(fpSize_list):
        MFPGEN = rdFingerprintGenerator.GetMorganGenerator(fpSize=fpSize, radius=r)

        # Fingerprints for train and test separately
        X_train = np.stack(df_train.Mol.apply(MFPGEN.GetCountFingerprintAsNumPy).values)
        y_train = df_train['boiling_point'].values

        X_test = np.stack(df_test.Mol.apply(MFPGEN.GetCountFingerprintAsNumPy).values)
        y_test = df_test['boiling_point'].values

        # Train the model
        model = RandomForestRegressor(n_estimators=100, random_state=random_seed)
        model.fit(X_train, y_train)

        # Predict on test set
        y_pred = model.predict(X_test)

        # Metrics
        mae = mean_absolute_error(y_test, y_pred)
        rmse = root_mean_squared_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)

        metrics_ood[(r, fpSize)] = {
            'rmse': rmse,
            'mae': mae,
            'r2': r2,
        }


  0%|          | 0/4 [00:00<?, ?it/s]

100%|██████████| 4/4 [00:03<00:00,  1.17it/s]


In [23]:
def display_metrics_table(metrics_dict):
    rows = []
    for (r, fpSize), metrics in metrics_dict.items():
        row = {
            'Radius': r,
            'Fingerprint Size': fpSize,
            'RMSE': metrics['rmse'],
            'MAE': metrics['mae'],
            'R2': metrics['r2']
        }
        rows.append(row)
    
    df = pd.DataFrame(rows)
    
    min_rmse = df['RMSE'].min()
    min_mae = df['MAE'].min()
    max_abs_r2 = df['R2'].max()

    def highlight_min(s, min_value, maximize=False):
        if maximize:
            return ['background-color: green' if abs(v) == min_value else '' for v in s]
        else:
            return ['background-color: green' if v == min_value else '' for v in s]

    styled_df = df.style\
        .apply(lambda s: highlight_min(s, min_rmse), subset=['RMSE'])\
        .apply(lambda s: highlight_min(s, min_mae), subset=['MAE'])\
        .apply(lambda s: highlight_min(s, max_abs_r2, maximize=True), subset=['R2'])\
        .format("{:.4f}", subset=['RMSE', 'MAE', 'R2'])

    return styled_df


In [24]:
display_metrics_table(metrics_ood)

,Radius,Fingerprint Size,RMSE,MAE,R2
0,0,256,58.2490,48.3190,-2.2063
1,0,512,58.2490,48.3190,-2.2063
2,0,1024,58.2490,48.3190,-2.2063
3,0,2048,58.2490,48.3190,-2.2063
4,0,4096,58.2490,48.3190,-2.2063
5,1,256,64.3495,55.5215,-2.9131
6,1,512,64.7788,56.0185,-2.9654
7,1,1024,64.3219,55.4895,-2.9097
8,1,2048,64.0253,55.1455,-2.8737
9,1,4096,64.2477,55.4035,-2.9007
